# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [38]:
# Importação dos pacotes para Analise
 
import pandas as pd
import seaborn as sns
import pprint
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

import warnings
warnings.simplefilter(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
#Carregar os dados para o google colab

uploaded = files.upload()

Saving dotz.txt to dotz.txt


In [0]:
# Importação dos dataset
df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Análise dos dados

In [0]:
#df['DESCRIÇÃO PARCEIRO'].str.slice(0,20)]

In [0]:
# Dicionario de balanceamento DEPARTAMENTO
qtd_dep_max = df['DEPARTAMENTO'].value_counts().max()
qtd_dep = qtd_dep_max - df['DEPARTAMENTO'].value_counts()
qtd_dep = dict(qtd_dep)

dep_ = df['DEPARTAMENTO'].value_counts() >= (qtd_dep_max // 2)
dep_ = dep_[dep_.isin([True])]
lista_dep_ = list(dep_.index)

for x in lista_dep_:
  qtd_dep.pop(x)
#qtd_dep

In [0]:
# Dicionario de balanceamento CATEGORIA
qtd_cat_max = df['CATEGORIA'].value_counts().max()
qtd_cat = qtd_cat_max - df['CATEGORIA'].value_counts()
qtd_cat = dict(qtd_cat)

cat_ = df['CATEGORIA'].value_counts() >= (qtd_cat_max / 2)
cat_ = cat_[cat_.isin([True])]
lista_cat_ = list(cat_.index)

for x in lista_cat_:
  qtd_cat.pop(x)
#qtd_cat

### Etapa de Preparação do Dados

In [0]:
# Variáveis importantes
pontuacao = ['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
# Palavras para retirar da análise
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct','und','das','no','ou', 
              'com','sem','gfa','jg','la','1','2','3','4','5','6','7','8','9','0','a','b','c','d','e',
              'f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','w','y','z']
stop_words_sub = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct','und','das',
                  'no','ou','gfa','jg','la','1','2','3','4','5','6','7','8','9','0']

# Tamanho da validação de teste
test_size = 0.001
random_state = 1
# Parametros do modelo PassiveAggressiveClassifier
#max_iter = 5000
#tol = 0.01
# Parametros do modelo LinearSVC
penalty = 'l1'
dual = False
tol = 0.001
max_iter = 4000
loss = 'hinge'
multi_class = 'crammer_singer'
# Interações no modelo PassiveAggressiveClassifier
#max_iter = 10000
#validation_fraction = 0.0001
#fit_intercept = True
#tol = 0.01
# Parametros do vetor CountVectorizer
ngram_range = (1, 2)
strip_accents = 'ascii'
# Parametros do vetor TfidfTransformer
use_idf = True

In [45]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')
  
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C


In [46]:
'''
# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

#Ajustando os departamentos
df['DEP_AJUSTADA'] = df['DEPARTAMENTO'].str.lower()

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].str.replace(x, ' ')

# Eliminando as stop words
df['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].apply(tokenize_df)
df.head()
'''

"\n# Função para retirar stop words\ndef tokenize_df(tokenized_words):\n  tokenized_words = word_tokenize(tokenized_words)\n  stop = [word for word in tokenized_words if word not in stop_words]\n  new_text = TreebankWordDetokenizer().detokenize(stop)\n  return new_text\n\n#Ajustando os departamentos\ndf['DEP_AJUSTADA'] = df['DEPARTAMENTO'].str.lower()\n\n# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.\nfor x in pontuacao:\n  df['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].str.replace(x, ' ')\n\n# Eliminando as stop words\ndf['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].apply(tokenize_df)\ndf.head()\n"

In [47]:
'''
# Definindo base de treino e teste

valida_test = pd.DataFrame(data=y_test3, columns=['SUB-CATEGORIA'])
idex_test = list(valida_test.index)
valida_train = pd.DataFrame(data=y_train3, columns=['SUB-CATEGORIA'])
idex_train = list(valida_train.index)
print('Tamanho da base de treino:', len(idex_train), 'Tamanho da base de teste:', len(idex_test))
'''

"\n# Definindo base de treino e teste\n\nvalida_test = pd.DataFrame(data=y_test3, columns=['SUB-CATEGORIA'])\nidex_test = list(valida_test.index)\nvalida_train = pd.DataFrame(data=y_train3, columns=['SUB-CATEGORIA'])\nidex_train = list(valida_train.index)\nprint('Tamanho da base de treino:', len(idex_train), 'Tamanho da base de teste:', len(idex_test))\n"

In [48]:
df[df['DESC_AJUSTADA'].str.contains("PEPPERONI")]

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
860,PIZZA Z/LAC GOURM BRASIL 380G PEPPERONI,PEPPERONI,PIZZA,PRATOS PRONTOS,PIZZA Z LAC GOURM BRASIL G PEPPERONI
15570,SALAME HANS PEPPERONI FAT kg,PEPPERONI,SALAME,FRIOS,SALAME HANS PEPPERONI FAT kg
21379,BISC SAL APERI AYMORE SALPET PIZZA PEPPERONI P...,APERITIVO SIMPLES,BISCOITO,MERCEARIA DOCE,BISC SAL APERI AYMORE SALPET PIZZA PEPPERONI PC G


### Etapa 1 - Classificando DEPARTAMENTO

In [49]:
#Criando a nova para classificar DEPARTAMENTO
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DESC_AJUSTADA']
base_target = base['DEPARTAMENTO']

cvt = CountVectorizer(ngram_range=ngram_range, strip_accents=strip_accents, stop_words=stop_words)
tfi = TfidfTransformer(use_idf=use_idf)
smt = SMOTE(qtd_dep, k_neighbors=3)
clf = LinearSVC(penalty=penalty, dual=dual, multi_class=multi_class, tol=tol, loss=loss, max_iter=max_iter) 
#clf = PassiveAggressiveClassifier(tol=tol, max_iter=max_iter, validation_fraction=validation_fraction, fit_intercept=fit_intercept)   

pipeline = Pipeline([('cvt', cvt), ('tfi', tfi)])
base_vetor = pipeline.fit_transform(base_data)
x_smote, y_smote = smt.fit_resample(base_vetor, base_target)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=test_size, random_state=random_state)

clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       1.00      1.00      1.00         4
                 AÇOUGUE       1.00      1.00      1.00         3
      BEBIDAS ALCOÓLICAS       1.00      1.00      1.00         5
  BEBIDAS NÃO ALCOÓLICAS       1.00      1.00      1.00         2
               BRINQUEDO       1.00      1.00      1.00         1
                 CALÇADO       1.00      1.00      1.00         2
      CAMA, MESA E BANHO       1.00      1.00      1.00         1
        CONGELADOS DOCES       1.00      1.00      1.00         3
     CONGELADOS SALGADOS       1.00      1.00      1.00         3
    CUIDADOS COM A BARBA       1.00      1.00      1.00         6
     CUIDADOS COM A CASA       1.00      1.00      1.00         2
  CUIDADOS COM A COZINHA       1.00      1.00      1.00         5
    CUIDADOS COM A ROUPA       1.00      1.00      1.00         5
 CUIDADOS COM MÃOS E PÉS       1.00      

In [50]:
'''
# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO DEPARTAMENTO

# Montando Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()) ])
text_clf = text_clf.fit(x_train, y_train)

# Valindando performance
predicted = text_clf.predict(x_test)
print('Performance com os paramentros default\n', np.mean(predicted == y_test))

# Melhores paramentros
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
              'clf__penalty': ('l1', 'l2'),
              'clf__multi_class': ('ovr', 'crammer_singer'),
              'clf__loss': ('hinge', 'squared_hinge'),
              'clf__dual': (True, False),
              'clf__tol': (1e-3, 1e-4),
              'clf__max_iter': (2000, 3000), }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(x_train, y_train)

print('Melhor score\n', gs_clf.best_score_)
print('Melhores parametros')
pprint.pprint(gs_clf.best_params_) 
'''

"\n# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO DEPARTAMENTO\n\n# Montando Pipeline\ntext_clf = Pipeline([('vect', CountVectorizer()),\n                     ('tfidf', TfidfTransformer()),\n                     ('clf', LinearSVC()) ])\ntext_clf = text_clf.fit(x_train, y_train)\n\n# Valindando performance\npredicted = text_clf.predict(x_test)\nprint('Performance com os paramentros default\n', np.mean(predicted == y_test))\n\n# Melhores paramentros\nparameters = {'vect__ngram_range': [(1, 2), (1, 3)],\n              'clf__penalty': ('l1', 'l2'),\n              'clf__multi_class': ('ovr', 'crammer_singer'),\n              'clf__loss': ('hinge', 'squared_hinge'),\n              'clf__dual': (True, False),\n              'clf__tol': (1e-3, 1e-4),\n              'clf__max_iter': (2000, 3000), }\n\ngs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)\ngs_clf = gs_clf.fit(x_train, y_train)\n\nprint('Melhor score\n', gs_clf.best_score_)\nprint('Melhores parametros')\npprint.pprint(gs_clf.best_param

In [51]:
# Predição em novos dados com o DF  de teste
base_novos = df2['DESC_AJUSTADA']
base_novos = pipeline.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(base_novos)

# Adicionando a coluna para o novo DF
df_dep = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_dep['DEPARTAMENTO'] = predicted_novos
#df_dep['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_dep.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO


In [52]:
'''
# Gerando dataframe para validação
def detokenize(texto):
  novo_texto = TreebankWordDetokenizer().detokenize(texto)
  return novo_texto

valida_dep = pd.DataFrame(data=y_test, columns=['DEPARTAMENTO'])
valida_dep['DEPARTAMENTO_PREDIÇÃO'] = predicted
valida_dep['DESCRIÇÃO'] = vetor.inverse_transform(x_test)
valida_dep['DESCRIÇÃO'] = valida_dep['DESCRIÇÃO'].apply(detokenize)
valida_dep['VALIDA'] = valida_dep['DEPARTAMENTO_PREDIÇÃO'] == valida_dep['DEPARTAMENTO']
valida_dep = valida_dep.query('VALIDA == False')

valida_departamento = valida_dep['DESCRIÇÃO'] + '     |' + valida_dep['DEPARTAMENTO'] + '    |' + valida_dep['DEPARTAMENTO_PREDIÇÃO']
valida_departamento = str(sorted(list(valida_departamento)))

# Gerar um arquivo com a lista de palavras unicas
arquivo = open('valida_departamento.txt', 'w')
arquivo.write(str(valida_departamento))
arquivo.close()
'''

"\n# Gerando dataframe para validação\ndef detokenize(texto):\n  novo_texto = TreebankWordDetokenizer().detokenize(texto)\n  return novo_texto\n\nvalida_dep = pd.DataFrame(data=y_test, columns=['DEPARTAMENTO'])\nvalida_dep['DEPARTAMENTO_PREDIÇÃO'] = predicted\nvalida_dep['DESCRIÇÃO'] = vetor.inverse_transform(x_test)\nvalida_dep['DESCRIÇÃO'] = valida_dep['DESCRIÇÃO'].apply(detokenize)\nvalida_dep['VALIDA'] = valida_dep['DEPARTAMENTO_PREDIÇÃO'] == valida_dep['DEPARTAMENTO']\nvalida_dep = valida_dep.query('VALIDA == False')\n\nvalida_departamento = valida_dep['DESCRIÇÃO'] + '     |' + valida_dep['DEPARTAMENTO'] + '    |' + valida_dep['DEPARTAMENTO_PREDIÇÃO']\nvalida_departamento = str(sorted(list(valida_departamento)))\n\n# Gerar um arquivo com a lista de palavras unicas\narquivo = open('valida_departamento.txt', 'w')\narquivo.write(str(valida_departamento))\narquivo.close()\n"

### Etapa 2 - Classificando CATEGORIA

In [53]:
#Criando a nova para classificar CATEGORIA
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DEPARTAMENTO'] + ' ' + base['DESC_AJUSTADA']
base_target = base['CATEGORIA']

#cvt = CountVectorizer(ngram_range=ngram_range, strip_accents=strip_accents, stop_words=stop_words)
#tfi = TfidfTransformer(use_idf=use_idf)
smt = SMOTE(qtd_cat, k_neighbors=2)
#clf = LinearSVC(penalty=penalty, dual=dual, multi_class=multi_class, tol=tol, loss=loss, max_iter=max_iter)    

pipeline = Pipeline([('cvt', cvt), ('tfi', tfi)])
base_vetor = pipeline.fit_transform(base_data)
x_smote, y_smote = smt.fit_resample(base_vetor, base_target)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=test_size, random_state=random_state)

clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                                   precision    recall  f1-score   support

                       ABSORVENTE       1.00      1.00      1.00         1
              ACESSÓRIOS DE BANHO       1.00      1.00      1.00         1
         ACESSÓRIOS DE LAVANDERIA       1.00      1.00      1.00         1
                   ACESSÓRIOS PET       1.00      1.00      1.00         1
                         ADOÇANTE       1.00      1.00      1.00         1
               AGUARDENTE/CACHAÇA       1.00      1.00      1.00         1
                          ALGODÃO       1.00      1.00      1.00         1
               ALIMENTO PARA AVES       1.00      1.00      1.00         1
              ALIMENTO PARA GATOS       1.00      1.00      1.00         1
                        AMACIANTE       1.00      1.00      1.00         1
        ANTISSÉPTICO BUCAL ADULTO       1.00      1.00      1.00         1
                APARELHO DE BARBA       1.00      1.00      1.00 

In [54]:
'''
# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO CATEGORIA

# Montando Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', PassiveAggressiveClassifier()) ])
text_clf = text_clf.fit(x_train, y_train)

# Valindando performance
predicted = text_clf.predict(x_test)
print('Performance com os paramentros default\n', np.mean(predicted == y_test))

# Melhores paramentros
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
              'vect__strip_accents': ('unicode', 'ascii'),
              'tfidf__use_idf': (True, False),
              'clf__tol': (0.01, 0.001),
              'clf__max_iter': (2000, 3000), }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(x_train, y_train)

print('Melhor score\n', gs_clf.best_score_)
print('Melhores parametros')
pprint.pprint(gs_clf.best_params_)  
'''

"\n# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO CATEGORIA\n\n# Montando Pipeline\ntext_clf = Pipeline([('vect', CountVectorizer()),\n                     ('tfidf', TfidfTransformer()),\n                     ('clf', PassiveAggressiveClassifier()) ])\ntext_clf = text_clf.fit(x_train, y_train)\n\n# Valindando performance\npredicted = text_clf.predict(x_test)\nprint('Performance com os paramentros default\n', np.mean(predicted == y_test))\n\n# Melhores paramentros\nparameters = {'vect__ngram_range': [(1, 2), (1, 3)],\n              'vect__strip_accents': ('unicode', 'ascii'),\n              'tfidf__use_idf': (True, False),\n              'clf__tol': (0.01, 0.001),\n              'clf__max_iter': (2000, 3000), }\n\ngs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)\ngs_clf = gs_clf.fit(x_train, y_train)\n\nprint('Melhor score\n', gs_clf.best_score_)\nprint('Melhores parametros')\npprint.pprint(gs_clf.best_params_)  \n"

In [55]:
# Predição em novos dados utilizando o df_dep
base_novos = df_dep['DEPARTAMENTO'] + ' ' + df_dep['DESC_AJUSTADA']

base_novos = pipeline.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(base_novos)

# Adicionando a coluna no novo DF
df_cat = df_dep
df_cat['CATEGORIA'] = predicted_novos
#df_cat['DESC_AJUSTADA_SUB'] = df_dep['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,ENFEITES DE NATAL


### Etapa 3 - Classificando SUB-CATEGORIA

In [0]:
lista_ajuste_fino = ['ÚMIDO','VINHO TINTO','ARTESANAIS','BALAS - MASTIGÁVEIS','BASE NATURAL',
                     'BÁSICA P','BÁSICO','CHOCOLATE','CHOCOLATE BARRA','CREMOSO','DELICADA',
                     'ERVAS','ESPAGUETE','ESPECIAIS','FRALDA GERIATRICA G','FRALDA GERIATRICA M',
                     'FRUTAS','GELEIA DIET','GLICERINADO','HIDRATANTE','INTERMEDIÁRIA P','OUTROS',
                     'PERFUMADO','RECHEADO','RESTAURADOR','SABORIZADO','SECO','TRADICIONAL',
                     'VESTIR P','VINHO BRANCO']

In [0]:
# Inserindo as SUB-CATEGORIA com baixa acurácia
max_sub = df['SUB-CATEGORIA'].value_counts().max()

for ajuste in lista_ajuste_fino:
  df_aux = df.rename(columns={"SUB-CATEGORIA": "SUBCATEGORIA"})
  df_aux = df_aux.query('SUBCATEGORIA in @ajuste')
  df_aux = df_aux.rename(columns={"SUBCATEGORIA": "SUB-CATEGORIA"})
  qtd_sub = df_aux.shape[0]

  while qtd_sub < max_sub:
    df_aux = df_aux.append(df_aux)
    qtd_sub = df_aux.shape[0]

  df = df.append(df_aux)

In [58]:
# Duplicando SUB-CATEGORIAS com apenas 1 produto
sub_ = df['SUB-CATEGORIA'].value_counts() == 1
sub_ = sub_[sub_.isin([True])]
lista_sub_ = list(sub_.index)
lista_sub_

df_aux = df.rename(columns={"SUB-CATEGORIA": "SUBCATEGORIA"})
df_aux = df_aux.query('SUBCATEGORIA in (@lista_sub_)')
df_aux = df_aux.rename(columns={"SUBCATEGORIA": "SUB-CATEGORIA"})
df = df.append(df_aux)
df.shape

(57185, 5)

In [0]:
# Dicionario de balanceamento SUB-CATEGORIA
qtd_sub_max = df['SUB-CATEGORIA'].value_counts().max()
qtd_sub = qtd_sub_max // 16 - df['SUB-CATEGORIA'].value_counts()
qtd_sub = dict(qtd_sub)

sub_ = df['SUB-CATEGORIA'].value_counts() >= (qtd_sub_max // 32)
sub_ = sub_[sub_.isin([True])]
lista_sub_ = list(sub_.index)

for x in lista_sub_:
  qtd_sub.pop(x)
#qtd_sub

In [60]:
# Criando a nova para classificar SUB-CATEGORIA retirando apenas números
base = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['CATEGORIA'] + ' ' + base['DESC_AJUSTADA']
base_target =  base['SUB-CATEGORIA']

#cvt = CountVectorizer(ngram_range=ngram_range, strip_accents=strip_accents, stop_words=stop_words)
#tfi = TfidfTransformer(use_idf=use_idf)
smt = SMOTE(qtd_sub, k_neighbors=1)
#clf = LinearSVC(penalty=penalty, dual=dual, multi_class=multi_class, tol=tol, loss=loss, max_iter=max_iter)    

pipeline = Pipeline([('cvt', cvt), ('tfi', tfi)])
base_vetor = pipeline.fit_transform(base_data)
x_smote, y_smote = smt.fit_resample(base_vetor, base_target)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=test_size, random_state=random_state)

clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                                precision    recall  f1-score   support

                    10 VOLUMES       1.00      1.00      1.00         1
              ABACATE ORGÂNICO       1.00      1.00      1.00         1
           ABACAXI COM HORTELÃ       1.00      1.00      1.00         1
                     ABOBRINHA       1.00      1.00      1.00         1
ACESSÓRIOS MAQUIAGEM - ESPONJA       1.00      1.00      1.00         1
                         ADUBO       1.00      1.00      1.00         1
               AEROSOL INSETOS       1.00      1.00      1.00         2
                       ALECRIM       1.00      1.00      1.00         1
                      ALFAVACA       1.00      1.00      1.00         1
                   ALHO FLOCOS       1.00      1.00      1.00         1
                ALHO TRITURADO       1.00      1.00      1.00         1
          ALMOFADA EXTRA FORTE       1.00      1.00      1.00         1
           ALTA PERFORMANCE E

In [61]:
# Predição em novos dados
base_novos = df_cat['CATEGORIA'] + ' ' + df_cat['DESC_AJUSTADA']

base_novos = pipeline.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(base_novos)

# Adicionando a coluna
df_sub = df_cat
df_sub['SUB-CATEGORIA'] = predicted_novos

# Novos Dados de teste incluindo o departamento
df_sub.head(10)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL,SAL DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,ENFEITES DE NATAL,BOLA DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,MERCEARIA DOCE,FARINÁCEO,MISTURAS
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ACESSÓRIOS MÃO E PÉ,ALICATE
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,PIZZA,PRESUNTO
5,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK M,DESCARTÁVEIS,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S XXH,LAVANDERIA,ORGANIZADOR,CESTO DE ROUPA
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI GR OVOS ESPAGUETE,MERCEARIA SALGADA,MASSA SECA,ESPAGUETE
9,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON ML,CUIDADOS COM O CABELO,CONDICIONADOR,TRADICIONAL


In [0]:
dotz = df_sub[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')

In [63]:
dotz.head(10)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL DO HIMALAIA,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,BOLA DE NATAL,ENFEITES DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MISTURAS,FARINÁCEO
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE,ACESSÓRIOS MÃO E PÉ
4,MEIA PIZZA PEPPERONI UN,PRESUNTO,PIZZA
5,JARRA ACASA VIDRO CURVES OJG378,JARRA/GARRAFA,UTENSÍLIOS DE VIDRO
6,FILME DE PVC CONSERV PACK 30M,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO DE ROUPA,ORGANIZADOR
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,ESPAGUETE,MASSA SECA
9,COND NOVEX OLEO OJON 300ML,TRADICIONAL,CONDICIONADOR


In [64]:
'''
pontuacao = ['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct','und','das',
              'no','ou','sem','gfa','jg','p/','la','1','2','3','4','5','6','7','8','9','0','a',
              'b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v',
              'x','w','y','z','kit','po','liq','cr','sh','pt','des','imp','lt','vd','az','esm',
              'cx','bc','lv','ad','gf','mix','rf','fr','bj','int','cd','cz','tp','ap','rs','caixa',
              'sao','bom','esp','extra','premium','pto','uso','sol','garrafa','brasil','pc','conj',
              'ph','gold','cj','mais','cor','facil','ice','classic','br','vm','tt','em','far','fix',
              'sc','dr','aco','nadir','romana','bauducco','original','pe','peso','pro','sl',
              'und','seda','tradicional','arcor','teka','bas','dec','fina','palmolive','neutro',
              'sec','peq','baw','brinox','ext','fd','up','waw','seara','sadia','del','oetker',
              'royal','ret','boni','black','ln','new','preta','corttex','blue','big','ou','co','ex',
              'paf','pif','box','bdj','bestway','cinza','dourada','gluten','plasvale','protex','ype',
              'fs','for','no','das','lg','me','pvc','refil','tirolez','sm','friboi','ipa','pr','st',
              'xg','xxg','mc','nr','cp','ox','pa','pq','me','unid','ana','bi','di','gr',
              'hereford','eg','daterra','grill','apim','distriboi','peca','tras','real','temp', #AÇOUGUE
              'tatu','elgolli','bassi','ancho','korin','organico','copacol','cord','simunovic', #AÇOUGUE
              'montana','aurora','vacuo','francezinha','germania','rainha','gourmet','gessner', #AÇOUGUE
              'perdigão','natupeixe','ferreira','ceratti','alegra','top','quality','dochef','atm', #AÇOUGUE
              'band','argus','halal','juliatto','corella','pamplona','promocao','peça','cubos', #AÇOUGUE
              'superfrango','regina','tulipa','millenium','angus','las','piedras','borb','brachola', #AÇOUGUE
              'swift','beff','paulista','osso','temperada','temperado','rech','palha','borboleta', #AÇOUGUE
              'capa','paz','pel','res','touc','beleco','organica','best','beef','granel','pele', #AÇOUGUE
              'maturatta','chorizo','nabrasa','dagranja','bd','macedo','sabor','chef','soberano', #AÇOUGUE
              'defumada','coz','porcionado','sulita','minga','lar','valesul','frxb','levida','agreco', #AÇOUGUE
              'desos','jagua','pdc','entrec','rica','nobre','avestruz','viva','reserva','parmes','iqf', #AÇOUGUE
              'el','golli','ivaipora','nho','bento','nova','gonzales','oriental','nat','bja','pamplona', #AÇOUGUE
              'ouro','temperatto','especial','joaquim','frescal','def','sulita','importada','importado', #AÇOUGUE
              'choice','usa','carinhas','bem','coated','canoa','cain','falani','forno','minas','noisett',
              'rus','serya','rusti','sticks','zuppa','crinkle','falani','fontini','fries','lutosa','mccain',
              'noisettes','saute','eleve','carone','demarchi','floretes','daucy','empanitos','auroggets',
              'petisco','perdigao','leoni','pinguin','brasa','mr','veggy','ecobras','org','superbom','texas',
              'geronimo','casa','rezende','sitio','miguel','cook','noiset','in','natu','wessel','atelie',
              'carone','sodebo']

# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')

# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

# Eliminando as stop words
df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].apply(tokenize_df)

# Listando arquivo
dotz = df.query('DEPARTAMENTO == "CONGELADOS SALGADOS"')
dotz = dotz['DESC_AJUSTADA'] + '     |' + dotz['DEPARTAMENTO'] + '    |' + dotz['CATEGORIA']
dotz = sorted(list(dotz))

# Gerar um arquivo com a lista de palavras unicas
arquivo = open('dotz.txt', 'w')
arquivo.write(str(dotz))
arquivo.close()
'''

'\npontuacao = [\'.\',\',\',\'-\',\'+\',\':\',\';\',\'&\',\'+\',\'/\',\'!\',\'#\',\'%\',\'(\',\')\',\'  \']\nstop_words = [\'em\',\'sao\',\'ao\',\'de\',\'da\',\'do\',\'para\',\'c\',\'kg\',\'un\',\'ml\',\'pct\',\'und\',\'das\',\n              \'no\',\'ou\',\'sem\',\'gfa\',\'jg\',\'p/\',\'la\',\'1\',\'2\',\'3\',\'4\',\'5\',\'6\',\'7\',\'8\',\'9\',\'0\',\'a\',\n              \'b\',\'c\',\'d\',\'e\',\'f\',\'g\',\'h\',\'i\',\'j\',\'k\',\'l\',\'m\',\'n\',\'o\',\'p\',\'q\',\'r\',\'s\',\'t\',\'u\',\'v\',\n              \'x\',\'w\',\'y\',\'z\',\'kit\',\'po\',\'liq\',\'cr\',\'sh\',\'pt\',\'des\',\'imp\',\'lt\',\'vd\',\'az\',\'esm\',\n              \'cx\',\'bc\',\'lv\',\'ad\',\'gf\',\'mix\',\'rf\',\'fr\',\'bj\',\'int\',\'cd\',\'cz\',\'tp\',\'ap\',\'rs\',\'caixa\',\n              \'sao\',\'bom\',\'esp\',\'extra\',\'premium\',\'pto\',\'uso\',\'sol\',\'garrafa\',\'brasil\',\'pc\',\'conj\',\n              \'ph\',\'gold\',\'cj\',\'mais\',\'cor\',\'facil\',\'ice\',\'classic\',\'br\',\'vm\',\'tt\',\'em\

In [65]:
'''
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'] + ' | ' + df['CATEGORIA'] + ' | ' + df['SUB-CATEGORIA']

# Listando arquivo

dotz = dict(df['DESC_AJUSTADA'].value_counts())

# Gerar um arquivo com a lista de palavras unicas
arquivo = open('dotz.txt', 'w')
arquivo.write(str(dotz))
arquivo.close()
'''

"\ndf['DESC_AJUSTADA'] = df['DEPARTAMENTO'] + ' | ' + df['CATEGORIA'] + ' | ' + df['SUB-CATEGORIA']\n\n# Listando arquivo\n\ndotz = dict(df['DESC_AJUSTADA'].value_counts())\n\n# Gerar um arquivo com a lista de palavras unicas\narquivo = open('dotz.txt', 'w')\narquivo.write(str(dotz))\narquivo.close()\n"